In [34]:
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import StrOutputParser
import joblib
import os
from langchain_community.llms import CTransformers
import time
from langchain.prompts import PromptTemplate


In [117]:
config = {
        'max_new_tokens': 500,  # Adjusted to potentially reduce response time
        'temperature': 0.7,     # Adjusted for more coherent responses
        'context_length' : 2048
}

    # Initialize the model
llm = CTransformers(
        model='models/llama-2-7b-chat.ggmlv3.q2_K.bin',
        model_type='llama',
        config=config
    )


In [118]:
import warnings
warnings.filterwarnings('ignore')

In [119]:
template = """
<s>[INST] <<SYS>>
You are a helpful AI assistant who has to act as advocate.
Answer based on the context provided. Don't answer unnecessarily if you dont find the context.
<</SYS>>
{context}
Question: {question}
Helpful Answer: [/INST]
"""

In [120]:
prompt = PromptTemplate.from_template(template)

In [131]:
reader = PdfReader('data/fine_tune_data.pdf')
# reader1= PdfReader('data/constitution_nowater.pdf')

In [132]:
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

# for i, page in enumerate(reader1.pages):
#     text = page.extract_text()
#     if text:
#         raw_text += text


In [133]:
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 350,
    chunk_overlap  = 20,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [134]:
embeddings_file = "./data/cases_genderviolence.joblib"
if os.path.exists(embeddings_file):
    embeddings = joblib.load(embeddings_file)
else:
    embeddings = HuggingFaceEmbeddings()
    joblib.dump(embeddings, embeddings_file)

In [135]:
vectorstore = Chroma.from_texts(texts, embeddings)

In [136]:
retriever = vectorstore.as_retriever()

In [137]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
)

In [138]:
query="My teacher keeps on making physical contact with me intentionally.What could i do?"

In [139]:
retriever.get_relevant_documents(query)


[Document(page_content='Chapter on Harassment, Section 223.  For the offence of Sexual Assault: Unwanted sexual contact, the punishment is Up to 10 years \nimprisonment and a fine of up to NPR 100,000 according to the sect ion of Muluki Ain (General Code), \nChapter on Sexual Offenses, Section 221.'),
 Document(page_content='Chapter on Harassment, Section 223.  For the offence of Sexual Assault: Unwanted sexual contact, the punishment is Up to 10 years \nimprisonment and a fine of up to NPR 100,000 according to the sect ion of Muluki Ain (General Code), \nChapter on Sexual Offenses, Section 221.'),
 Document(page_content='For the offence of Sexual Harassment: Unwelcome sexual advances, requests for sexual favors, and \nother verbal or physical harassment of a sexual nature, the punishment is Up to 3 years impri sonment \nand a fine of up to NPR 30,000 according to the section of Sexual Harassment at Workplace Prevention \nAct, 2015, Section 5.'),
 Document(page_content='For the offence

In [140]:
start_time = time.time()
print(chain.invoke(query))
end_time = time.time()
print(f"Response time: {end_time - start_time:.2f} seconds")

I'm so sorry to hear that you're going through this difficult situation. It's important to remember that no one has the right to touch or make physical contact with you without your consent, and it's completely okay to stand up for yourself in situations like this. Here are some steps you can take:

1. Tell a trusted adult: If you feel uncomfortable talking to your teacher directly, you can tell a trusted adult such as a school administrator, counselor, or parent about the situation. They can provide support and help address the issue.
2. Seek support from classmates: You may also want to talk to other students in your class who may have experienced similar situations. It can be helpful to know that you're not alone and that there are people who care about you and want to help.
3. Consider reporting the incident: If you feel comfortable, you can report the incident to the appropriate authorities. In many places, there are laws in place to protect people from sexual harassment and assau